# AWS Polly와 Translate를 사용한 음성 변환 예제

### 코드 설명

1. **번역 및 음성 변환 함수**: 먼저, AWS Translate를 사용하여 한국어 텍스트를 영어로 번역하는 함수와 AWS Polly를 사용하여 텍스트를 음성으로 변환하는 함수를 구현합니다.

2. **시간 측정**: Python의 `time` 모듈을 사용하여 번역 및 음성 변환 과정에 걸리는 시간을 측정합니다.

3. **실행**: 한국어 텍스트를 입력으로 받아 영어로 번역하고 음성 파일을 생성하는 과정, 그리고 한국어 텍스트를 한국어 음성으로 변환하는 과정을 실행하고 각각의 시간을 출력합니다.
4. 

In [12]:
!pip install boto3

zsh:1: command not found: pip


AWS Translate와 Polly 서비스에 접근하기 위해 boto3 클라이언트를 생성합니다.

In [41]:
import boto3
from contextlib import closing
import time
import uuid

# AWS Translate와 Polly 서비스에 접근하기 위해 boto3 클라이언트를 생성합니다.
translate_client = boto3.client('translate')
polly_client = boto3.client('polly')

#### 1. 번역 및 음성 변환 함수
이 섹션에서는 AWS Translate와 AWS Polly 서비스를 사용하기 위한 두 가지 주요 함수를 정의합니다.

- `translate_text` 함수는 주어진 텍스트를 한 언어에서 다른 언어로 번역합니다. 이 예제에서는 한국어를 영어로 번역합니다.
- `text_to_speech` 함수는 주어진 텍스트를 음성으로 변환합니다. 이 함수는 번역된 텍스트 또는 원본 텍스트를 사용하여 음성 파일을 생성할 수 있습니다.

In [38]:
def translate_text(text, source_language_code, target_language_code):
    """
    AWS Translate를 사용하여 주어진 텍스트를 번역합니다.
    :param text: 번역할 원본 텍스트
    :param source_language_code: 원본 텍스트의 언어 코드 (예: 'ko' for Korean)
    :param target_language_code: 대상 언어 코드 (예: 'en' for English)
    :return: 번역된 텍스트
    """
    # 번역 요청 및 응답
    response = translate_client.translate_text(
        Text=text,
        SourceLanguageCode=source_language_code,
        TargetLanguageCode=target_language_code
    )
    # 번역된 텍스트 반환
    return response['TranslatedText']

In [42]:
def text_to_speech(text, language_code, voice_id, format='mp3'):
    """
    AWS Polly를 사용하여 주어진 텍스트를 음성으로 변환합니다.
    :param text: 음성으로 변환할 텍스트
    :param language_code: 텍스트의 언어 코드 (예: 'ko-KR' for Korean)
    :param voice_id: 사용할 음성의 ID (예: 'Seoyeon' for a Korean female voice)
    :param format: 출력 파일 형식 (예: 'mp3')
    :return: 생성된 음성 파일의 경로
    """
    # 음성 변환 요청
    response = polly_client.synthesize_speech(
        Text=text,
        OutputFormat=format,
        LanguageCode=language_code,
        VoiceId=voice_id
    )

    # 음성 파일 저장
    if "AudioStream" in response:
        with closing(response["AudioStream"]) as stream:
            # 고유한 파일명 생성
            unique_filename = f'speech_{uuid.uuid4()}.{format}'
            with open(unique_filename, 'wb') as file:
                file.write(stream.read())
            return unique_filename
    else:
        return None

#### 2. 시간 측정 및 음성 파일 재생
이 섹션에서는 번역 및 음성 변환 작업의 실행 시간을 측정합니다. Python의 `time` 모듈을 사용하여 각 작업의 시작과 종료 시간을 기록하고, 이를 통해 전체 소요 시간을 계산합니다. 이를 통해 AWS 서비스의 성능을 평가할 수 있습니다.

이 코드는 AWS Polly를 사용하여 생성된 두 개의 음성 파일을 재생합니다. 첫 번째 파일은 한국어 텍스트를 영어로 번역한 후 생성된 영어 음성 파일이며, 두 번째 파일은 원본 한국어 텍스트를 한국어 음성으로 변환한 파일입니다.

- `english_audio_file_path` 변수에는 영어 음성 파일의 경로가 저장되어 있습니다.
- `korean_audio_file_path` 변수에는 한국어 음성 파일의 경로가 저장되어 있습니다.

IPython의 `Audio` 클래스를 사용하여 이 파일들을 재생할 수 있습니다. 이를 통해 생성된 음성의 품질과 정확도를 직접 확인할 수 있습니다.

In [44]:
from IPython.display import Audio

# 테스트 텍스트
korean_text = "안녕하세요, AWS Polly를 사용하여 음성 변환을 테스트합니다."

# 한국어 -> 영어 번역 및 음성 변환
start_time = time.time()
translated_text = translate_text(korean_text, 'ko', 'en')
english_audio = text_to_speech(translated_text, 'en-US', 'Joanna')
end_time = time.time()
print("한국어 -> 영어 번역 및 음성 변환 시간:", end_time - start_time, "초")

# 영어 음성 파일 재생
english_audio_file_path = english_audio  # AWS Polly를 통해 생성된 영어 음성 파일 경로
display(Audio(english_audio_file_path))

한국어 -> 영어 번역 및 음성 변환 시간: 0.09337401390075684 초


In [45]:

# 한국어 -> 한국어 음성 변환
start_time = time.time()
korean_audio = text_to_speech(korean_text, 'ko-KR', 'Seoyeon')
end_time = time.time()
print("한국어 -> 한국어 음성 변환 시간:", end_time - start_time, "초")

# 한국어 음성 파일 재생
korean_audio_file_path = korean_audio  # AWS Polly를 통해 생성된 한국어 음성 파일 경로
display(Audio(korean_audio_file_path))

한국어 -> 한국어 음성 변환 시간: 0.13178181648254395 초
